# Build a medium size KG from a CSV dataset

OH: "If you build it, they will SPARQL"

First, let's initialize the KG object the same as we did in an earlier notebook:

In [1]:
import kglab

namespaces = {
    "wtm": "http://purl.org/heals/food/",
    "ind": "http://purl.org/heals/ingredient/",
    }

kg = kglab.KnowledgeGraph(
    name = "A recipe KG example based on Food.com",
    base_uri = "https://www.food.com/recipe/",
    language = "en",
    namespaces = namespaces,
    )

Next, we'll define a dictionary that maps (somewhat magically) from strings (i.e., "labels") to ingredients defined in the <http://purl.org/heals/ingredient/> vocabulary:

In [2]:
common_ingredient = {
    "water": kg.get_ns("ind").Water,
    "salt": kg.get_ns("ind").Salt,
    "pepper": kg.get_ns("ind").BlackPepper,
    "black pepper": kg.get_ns("ind").BlackPepper,
    
    "baking powder": kg.get_ns("ind").BakingPowder,
    "baking soda": kg.get_ns("ind").BakingSoda,

    "vanilla": kg.get_ns("ind").VanillaExtract,
    "vanilla extract": kg.get_ns("ind").VanillaExtract,

    "butter": kg.get_ns("ind").Butter,
    "milk": kg.get_ns("ind").CowMilk,
    "egg": kg.get_ns("ind").ChickenEgg,
    "eggs": kg.get_ns("ind").ChickenEgg,

    "sugar": kg.get_ns("ind").WhiteSugar,
    "brown sugar": kg.get_ns("ind").BrownSugar,
    "honey": kg.get_ns("ind").Honey,

    "flour": kg.get_ns("ind").AllPurposeFlour,
    "all-purpose flour": kg.get_ns("ind").AllPurposeFlour,
    "whole wheat flour": kg.get_ns("ind").WholeWheatFlour,

    "olive oil": kg.get_ns("ind").OliveOil,
    "vinegar": kg.get_ns("ind").AppleCiderVinegar,

    "onion": kg.get_ns("ind").Onion,
    "onions": kg.get_ns("ind").Onion,
    "garlic": kg.get_ns("ind").Garlic,
    "garlic clove": kg.get_ns("ind").Garlic,
    "garlic cloves": kg.get_ns("ind").Garlic,
    "cabbage": kg.get_ns("ind").Cabbage,
    "carrot": kg.get_ns("ind").Carrot,
    "carrots": kg.get_ns("ind").Carrot,
    "celery": kg.get_ns("ind").Celery,
    "potato": kg.get_ns("ind").Potato,
    "potatoes": kg.get_ns("ind").Potato,
    "tomato": kg.get_ns("ind").Tomato,
    "tomatoes": kg.get_ns("ind").Tomato,
}

Now load the dataset of recipes into a dataframe:

In [3]:
import pandas as pd

df = pd.read_csv("dat/recipes.csv")
df.head()

,id,name,minutes,tags,description,ingredients
0,164636,1 1 1 tempura batter,5,"['15-minutes-or-less', 'time-to-make', 'course...","i use this everytime i make onion rings, hot p...","['egg', 'flour', 'water']"
1,189437,40 second omelet,25,"['30-minutes-or-less', 'time-to-make', 'course...","you'll need an ""inverted pancake turner"" for t...","['eggs', 'water', 'butter']"
2,19104,all purpose dinner crepes batter,90,"['weeknight', 'time-to-make', 'course', 'main-...",this basic crepe recipe can be used for all yo...,"['eggs', 'salt', 'flour', 'milk', 'butter']"
3,284518,another perfect poached egg,13,"['15-minutes-or-less', 'time-to-make', 'course...",there's already a couple of recipes on here fo...,"['eggs', 'water', 'vinegar', 'salt']"
4,327593,anytime crepes,8,"['15-minutes-or-less', 'time-to-make', 'course...","from my friend linda, this is an oh-so-easy-an...","['egg', 'milk', 'whole wheat flour']"


Then iterate over the rows in the dataframe, representing a new recipe in the KG for each row:

In [4]:
import rdflib as rdf
from rdflib.namespace import RDF, XSD

for index, row in df.iterrows():
    recipe_id = row["id"]
    node = rdf.URIRef("https://www.food.com/recipe/{}".format(recipe_id))
    kg.add(node, RDF.type, kg.get_ns("wtm").Recipe)

    recipe_name = row["name"]
    kg.add(node, kg.get_ns("skos").definition, rdf.Literal(recipe_name))
    
    cook_time = row["minutes"]
    kg.add(node, kg.get_ns("wtm").hasCookTime, rdf.Literal(cook_time, datatype=XSD.integer))
    
    ind_list = eval(row["ingredients"])

    for ind in ind_list:
        ingredient = ind.strip()
        ingredient_obj = common_ingredient[ingredient]
        kg.add(node, kg.get_ns("wtm").hasIngredient, ingredient_obj)

The graph has grown by a couple orders of magnitude, so it's visualization should be more interesting now.
Let's take a look:

In [5]:
VIS_STYLE = {
    "wtm": {
        "color": "orange",
        "size": 20,
    },
    "ind":{
        "color": "blue",
        "size": 35,
    },
}

g = kg.vis_pyvis(notebook=True, style=VIS_STYLE)
g.barnes_hut()
g.show("tmp.html")

Given the defaults for this kind of visualization, there's likely a dense center mass of orange (recipes) at the center, with a close cluster of common ingredients (dark blue), surrounded by less common ingredients and cooking times (light blue).

Let's serialize the KG to a local file:

In [6]:
kg.save_ttl("tmp.ttl")

While we're at this, let's serialize in each of the three formats that we've been using, to compare the relative sizes for a medium-sized KG:

In [7]:
kg.save_jsonld("tmp.jsonld")
kg.save_parquet("tmp.parquet")

In [8]:
import pandas as pd
import os

file_paths = ["tmp.jsonld", "tmp.ttl", "tmp.parquet"]
file_sizes = [os.path.getsize(file_path) for file_path in file_paths]

df = pd.DataFrame({"file_path": file_paths, "file_size": file_sizes})
df

,file_path,file_size
0,tmp.jsonld,74376
1,tmp.ttl,38613
2,tmp.parquet,9119


Notice the relative sizes?
[Parquet](https://parquet.apache.org/) provides for compression in a way that works well with RDF.
The same KG stored as a Parquet file is less than 13% the size of the same KG stored as JSON-LD.